# "Will it snow tomorrow?" - The time traveler asked
The following dataset contains climate information form over 9000 stations accross the world. The overall goal of these subtasks will be to predict whether it will snow tomorrow 13 years ago. So if today is 2022.02.15 then the weather we want to forecast is for the date 2009.02.16. You are suppsed to solve the tasks using Big Query, which can be used in the Jupyter Notebook like it is shown in the following cell. For further information and how to used BigQuery in Jupyter Notebook refer to the Google Docs. 

The goal of this test is, to test your coding knowledge in Python, BigQuery and Pandas as well as your understanding of Data Science. If you get stuck at the first part, you can use the replacement data provided in the second part

### Summary
To solve the task given above, we roughly follow the following procedure: 

1. Load relevant data & apply filters mentioned in the task descriptions
2. Inspect data and select features to be used for prediction
3. Split data and train a selection of classifiers on the training set
4. Use the best performing model to predict whether it is snowing on target day

**Loading Bigquery**

In [88]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


## Part 1

### 1. Task
Change the date format to 'YYYY-MM-DD' and select the data from 2005 till 2009 for station numbers including and between 725300 and 726300 , and save it as a pandas dataframe. Note the maximum year available is 2010. 

For the bigquery magic decorator to work we need to define a GCP project. We use *learnings-coding-challenge*, the project created for this task. By putting a name after the decorator, the query result is automatically saved into a pandas.DataFrame object.

We retrieve all columns that fulfill the conditions posed in the task and create an additional column called *date* that contains the date in *'Y-M-D'* format. The *BETWEEN* operator is inclusive and thus stations 725300 and 726300 are included in the table returned. 

In [89]:
%%bigquery df --project learnings-coding-challenge
SELECT 
*, 
DATE(year, month, day) AS date
FROM `bigquery-public-data.samples.gsod`
WHERE year BETWEEN 2005 AND 2009
AND station_number BETWEEN 725300 AND 726300

Downloading: 100%|██████████| 377784/377784 [00:07<00:00, 51321.53rows/s]


### 2. Task 
**From here want to work with the data from all stations 725300 to 725330 that have information from 2005 till 2009.**

From here continue using Python - more specifically using pandas to manipulate the DataFrame we just retrieved.

In [90]:
#only keep data where station_number is within given range
relevant_stations = df[(df['station_number']>725299)&(df['station_number']<725331)]
#only keep data of stations that have data in each year
#we have five years
years = 5
sn_years = relevant_stations[['station_number', 'year']].groupby(['station_number']).nunique()
#only keep the stations where count of years == years
stations_all_years = list(sn_years[sn_years['year']==years].index)
stations_cleaned = relevant_stations[relevant_stations['station_number'].isin(stations_all_years)]

In our case, all stations within the range of station numbers we are interested in appear in each of the relevant years.

**Do a first analysis of the remaining dataset, clean or drop data depending on how you see appropriate.**

Feature selection is an important step to improve a model's performance. Given the time constraint and the fact that machine learning estimators used further below can handle high-dimensional data better than linear estimators such as Logistic Regression, we restrict ourselves to a few simple steps here. 

We start by investigating the percentage of missing observations across all features. Below the percentage of missings of each available variable is printed.

In [91]:
#what's percentage of NaN for each feature?
missing_pct = stations_cleaned.isnull().sum() * 100 / len(stations_cleaned)
print(missing_pct)

station_number                          0.000000
wban_number                             0.000000
year                                    0.000000
month                                   0.000000
day                                     0.000000
mean_temp                               0.000000
num_mean_temp_samples                   0.000000
mean_dew_point                          0.011032
num_mean_dew_point_samples              0.011032
mean_sealevel_pressure                 10.138452
num_mean_sealevel_pressure_samples     10.138452
mean_station_pressure                  94.522588
num_mean_station_pressure_samples      94.522588
mean_visibility                         0.016548
num_mean_visibility_samples             0.016548
mean_wind_speed                         0.027580
num_mean_wind_speed_samples             0.027580
max_sustained_wind_speed                0.055160
max_gust_wind_speed                    36.742236
max_temperature                         0.011032
max_temperature_expl

We see that multiple variables only contain missing values in the dataframe we have created by applying the imposed restrictions. Hence, we can remove these variables as well as other variables that have a high percentage of missing values. We set the treshold to 90% percent (i.e. observations with less than 10% missings are kept). 

In [92]:
#only keep variables with more than 90% of observations (just a random threshold really)
keep_cols = list(missing_pct[missing_pct<10].index)
low_missings = stations_cleaned[keep_cols]
print(keep_cols)

['station_number', 'wban_number', 'year', 'month', 'day', 'mean_temp', 'num_mean_temp_samples', 'mean_dew_point', 'num_mean_dew_point_samples', 'mean_visibility', 'num_mean_visibility_samples', 'mean_wind_speed', 'num_mean_wind_speed_samples', 'max_sustained_wind_speed', 'max_temperature', 'max_temperature_explicit', 'total_precipitation', 'fog', 'rain', 'snow', 'hail', 'thunder', 'tornado', 'date']


Next, we can rule out some features by taking a closer look at the documentation of the data (found here: https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=samples&t=gsod&page=table&_ga=2.176688416.426819128.1655642879-296738449.1655457946&project=learnings-coding-challenge).
Some features contained in the data are meta-data that are more than unlikely to contain valuable information that helps us to predict whether it will snow. For example, *num_mean_temp_samples* describes how many observations have been used to calculate the mean temperature that day. While the temperature itself is of obvious interest, this number is unlikely to bear any relevant information. This is true for any such variable (all contain 'sample' in their name) and thus we can remove them from the data.

Additionally, we remove the columns related to max_temperature as the documentation states that "*the time that this value is reported differs by country and region, so this value will sometimes not be the maximum for the calendar day."* This measurement error can bias our results severly as we can expect temperature to be a strong predictor of the snowing conditions.

In [93]:
#all columns containing the words 'sample' or 'max_temperature' can be removed 
sample_cols = [col for col in low_missings.columns if 'sample' in col]
max_temp_cols = [col for col in low_missings.columns if 'max_temp' in col]
final = low_missings.drop(sample_cols+max_temp_cols, axis = 1)

Although we have removed features with a high percentage of missings, we still have observations in our data that contain missing values for specific variables. In this case, we simply remove all rows with missing values in any feature as we end up with a reasonable amount of data. For notes on a potentially better approach (imputation) see the *Notes on Improvement* below.

To make sure that we are not deleting observations in which 

In [94]:
snow_balance = {}
for feat in final.columns:
    share_missing_feat = final[(final[feat].isnull())&(final['snow']==1)].shape[0]
    share_existing_feat = final[(final[feat].isnull()==False)&(final['snow']==1)].shape[0]
    snow_balance[feat]=(share_missing_feat, share_existing_feat)
print(snow_balance)

{'station_number': (0, 2142), 'wban_number': (0, 2142), 'year': (0, 2142), 'month': (0, 2142), 'day': (0, 2142), 'mean_temp': (0, 2142), 'mean_dew_point': (0, 2142), 'mean_visibility': (0, 2142), 'mean_wind_speed': (2, 2140), 'max_sustained_wind_speed': (3, 2139), 'total_precipitation': (55, 2087), 'fog': (0, 2142), 'rain': (0, 2142), 'snow': (0, 2142), 'hail': (0, 2142), 'thunder': (0, 2142), 'tornado': (0, 2142), 'date': (0, 2142)}


In [95]:
final = final.dropna()

**Notes on improvement:** Here are some thoughts on how preprocessing and the feature selection could be improved:

- impute missing values
    - right now we are losing observations because they do not contain information on a small subset of features; we could improve this by imputing these missing values 
    - I have no experience in imputation yet and cannot state anything reliably on the effects this might have on predictions but e.g. using mean imputation for weather data could bias our results in seasonal regions; looking at the variance of these features first could help assess how well imputation approaches might work or bias the results (?)
- inspect correlation of features with outcome to assess relevance of feature
- run a simple model (e.g. Lasso Regression, (pruned) Decision Tree/Random Forest) that allows for feature selection 
    - e.g. we could run a Lasso Regression on a random (bootstrapped) subsample of the data to retrieve information on which variables are relevant to include


### 3. Task
**Now it is time to split the data, into a training, evaluation and test set. As a reminder, the date we are trying to predict snow fall for is the following, and hence should constitute your test set.**

In [96]:
import datetime as dt
#save test date to create test set in next cell
test_date = str(dt.datetime.today()- dt.timedelta(days=13*365)).split(' ')[0]
#turn test_date into datetime object
test_date = dt.datetime.strptime(test_date, '%Y-%m-%d').date()
print(test_date)

2009-06-24


First, we define our outcome and some lists containing column names that we need for further data manipulation but should not be included as features in our prediction. Namely, these are meta information such as the date column, year, day and the two columns containing station numbers.

In [97]:
outcome = 'snow'
meta_cols = ['year', 'day', 'month', 'date', 'station_number', 'wban_number']
#transform the outcome variable from bool to integer 
final[outcome]=final[outcome].astype(int)

Below I split up the data into training, evaluation and test set. We start with the test set by only keeping observations on *test_date* (as of writing: 2009-06-23).

Next, we split up the pandas.DataFrame object into X and y, where y is the outcome variable to be predicted. X should not contain variables such as the station number and features related to the date (except month). We exclude these variables as they are unlikely to contain any relevant information that helps predict the likelihood of snow.

In [98]:
test = final[final['date']==test_date]
X_test = test.drop([outcome]+meta_cols, axis = 1)
y_test = test[outcome]

Now, we turn to splitting up the remaining observations into a training and an evaluation set. The latter will be used to fine-tune models and decide which model to use for final predictions. Similarly to the test set, columns without relevant information are removed.

In [99]:
#import necessary modules
from sklearn.model_selection import train_test_split

In [100]:
#ignore test date when creating training and evaluation set and remove irrelevant features
X = final[final['date']!=test_date].drop([outcome]+meta_cols, axis = 1)
y = final[final['date']!=test_date][outcome]
#split up into train and evaluation set 
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size = 0.1)

## Part 2
**If you made it up to here all by yourself, you can use your prepared dataset to train an Algorithm of your choice to forecast whether it will snow on the following date for each station in this dataset:**

In [101]:
import datetime as dt

str(dt.datetime.today()- dt.timedelta(days=13*365)).split(' ')[0]

'2009-06-24'

You are allowed to use any library you are comfortable with such as sklearn, tensorflow keras etc. 
If you did not manage to finish part one feel free to use the data provided in 'coding_challenge.csv' Note that this data does not represent a solution to Part 1. 

**Modules**

In [102]:
#model training and evaluation
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score

#plotting
import plotly.express as px

**Functions**

The following cell defines functions that are used to train models and evaluate their performance. For more details on what each function does, what inputs they use and what they return, see respective docstring.

In [107]:
#define functions for training models
def CV_fit(est, X, y, params):
    '''
    Use cross-validation grid search to find "best" estimator given a set of parameters.
    
    *est: sklearn estimator 
    *X: array-like
        Contains features used for prediction.
    *y: 1D array-like+
        Contains outcome to be predicted, must be binary (!)

    Returns: GridSearchCV.best_estimator_
    '''
    #first check if outcome supplied is binary 
    #! not yet working correctly
    # if ((y==0) | (y==1)).all()==False:
    #     raise ValueError(f'y must be binary.')
    gs_cv = GridSearchCV(est, param_grid = params)
    gs_cv.fit(X, y)
    
    return gs_cv.best_estimator_

def eval_predict(true, pred):
    '''
    Calculate MSE and F1-Score. Print and return results.
    '''
    mse = np.mean((pred-true)**2)
    #need to set zero_division to 1 in our case as test set only contains 0s
    f1 = f1_score(true, pred, zero_division = 1)
    
    res_str = f'MSE: {mse} \n F1-Score: {f1}'
    print(res_str)
    
    return mse, f1

### Model Training

To predict the snowing conditions on the target date we train a random forest classifier. Its advantage is its ease of implementation (time-efficient) and the fact that it can handle unscaled data well. Moreover, random forests implement non-linear estimation without pre-defining the degrees of interaction or polynomials (as would be necessary for linear classifiers for example) and it has a proven track-record of performing quite well as an out-of-the-box algorithm. Implementation of more sophisticated techniques would be subject to a more time-demanding procedure.

Each model is then evaluated based on the evaluation set and the final prediction of the test data is made based upon the best model. Although it might seem better for this specific prediction to use the test set to choose the best model, the fact that the test set only contains one observation makes the evaluation prone to result in a model too specifically focused on the test set.

**Random Forest Classifier**

In [104]:
#start with a simple grid search random forest classifier and use all available features 
#set up parameter dictionary 
params = {'max_depth': range(2, 10, 5), 'min_samples_split': range(10, 40, 5), 'min_samples_leaf': range(25, 50, 5)}
#get best RFC
RFC = RandomForestClassifier()
best_RFC = CV_fit(est = RFC, X = X_train, y = y_train, params = params)
#predict y_eval using fitted model
y_eval_RFC = best_RFC.predict(X_eval)

### Model Evaluation

In [108]:
#calculate evaluation metrics
mse_RFC, f1_RFC = eval_predict(y_eval, y_eval_RFC)
#bind metrics into a dataframe
metrics_dict = {'model': 'RFC', 
                'MSE': mse_RFC,
                'F1': f1_RFC}
metrics_df = pd.DataFrame.from_dict(metrics_dict, orient = 'index')

MSE: 0.0 
 F1-Score: 1.0


These results are quite odd as the model performs perfectly on the evaluation set. Usually, this might suggest problems of overfitting but the optimal *max_depth* parameter oafter cross-validation is 2, i.e. trees are extremely shallow. Taking a look at the feature importace (mean decrease in impurity by splitting on this feature across all trees) we see that features such as temperature and wind speed do not play any role in predicting the weather outcome. 

In [116]:
print([(feat, imp) for feat, imp in zip(X_train.columns,best_RFC.feature_importances_)])

[('mean_temp', 0.0), ('mean_dew_point', 0.0), ('mean_visibility', 0.010332645634549532), ('mean_wind_speed', 0.0), ('max_sustained_wind_speed', 0.0), ('total_precipitation', 0.0029535120466042206), ('fog', 0.27896161438030476), ('rain', 0.18194958970608957), ('hail', 0.14973876242463618), ('thunder', 0.16606387580781573), ('tornado', 0.21)]


Another reason might be the extremely selective subsample we are using. In case weather stations with similar numbers are close to each other, our model is likely to not be very generalizable to predicting snowing conditions around the globe but rather only in one specific area. 

To test this hypothesis, we create a new evaluation set which contains stations with numbers between 726000 and 726284 and evaluate the models performance for these stations.

In [119]:
#create a new eval set
new_eval = df[(df['station_number']>726000)&(df['station_number']<726284)]
#only keep stations that appear in 2005 to 2009
years = 5
sn_years_new_eval = new_eval[['station_number', 'year']].groupby(['station_number']).nunique()
#only keep the stations where count of years == years
stations_all_years_new_eval = list(sn_years_new_eval[sn_years_new_eval['year']==years].index)
new_eval = new_eval[new_eval['station_number'].isin(stations_all_years_new_eval)]
new_eval = new_eval[keep_cols]
new_eval = new_eval.drop(max_temp_cols+sample_cols, axis = 1)
new_eval = new_eval.dropna()
#then predict new evaluation set outcomes and measure performance
y_new_eval = new_eval[outcome].astype(int)
X_new_eval = new_eval.drop([outcome]+meta_cols, axis = 1)
y_new_eval_RFC = best_RFC.predict(X_new_eval)
mse_new, f1_new = eval_predict(y_new_eval, y_new_eval_RFC)


MSE: 0.0 
 F1-Score: 1.0
0.0 1.0


### Final Prediction

The evaluation of the predictor has shown it to be performing outstandingly well and we now predict the snowing conditions in the test set, i.e. on the target date.

In [127]:
y_test_RFC = best_RFC.predict(X_test)
eval_predict(y_test, y_test_RFC)

MSE: 0.0 
 F1-Score: 0.0


/Users/llccf/OneDrive/Dokumente/Bewerbungen/CodingChallenges/7Learnings/venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


(0.0, 0.0)